# 1. Collection des données

## 1.1 Installation des dépendances

In [ ]:
pip install labelme tensorflow opencv-python matplotlib albumentations

## 1.2 Collection des images

In [ ]:
import os
import time
import uuid
import cv2

In [ ]:
IMAGES_PATH = os.path.join('data','images')
RAW_IMAGES_PATH = os.path.join('data','raw_images')

NUMBER_OF_IMAGES = 30

In [ ]:
cap = cv2.VideoCapture(0)
time.sleep(5)

for i in range(NUMBER_OF_IMAGES):
    print(f"Capture de l'image {i}")
    time.sleep(2)
    
    success, frame = cap.read()
    if not success:
        print("Impossible de capturer l'image")
        break

    image_path = os.path.join(RAW_IMAGES_PATH, f'{uuid.uuid1()}.jpg')
    
    cv2.imwrite(image_path, frame)
    cv2.imshow('frame', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'): 
        break

cap.release()
cv2.destroyAllWindows()

## 1.3 Annotation des images avec LabelMe

In [ ]:
!labelme

# 2. Review du Dataset


## 2.1 Importation des deps

In [32]:
import tensorflow as tf
import json
import numpy as np
from matplotlib import pyplot as plt

## 2.2 Limiter la memoire du GPU

In [ ]:
# Avoid OOM errors by setting GPU Memory Consumption Growth
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [ ]:
tf.config.list_physical_devices('GPU')

## 2.3 Charger les images dans le pipeline tensorflow

Sur Windows:

In [ ]:
images = tf.data.Dataset.list_files('data\\images\\*.jpg')

Sur macOS:

In [ ]:
images = tf.data.Dataset.list_files('data/images/*.jpg')

In [ ]:
images.as_numpy_iterator().next()

In [ ]:
def load_image(x): 
    byte_img = tf.io.read_file(x)
    img = tf.io.decode_jpeg(byte_img)
    return img

In [ ]:
images = images.map(load_image)

In [ ]:
images.as_numpy_iterator().next()

In [ ]:
type(images)

## 2.4 Afficher l'image dans Matplotlib

In [ ]:
image_generator = images.batch(4).as_numpy_iterator()

In [ ]:
plot_images = image_generator.next()

In [ ]:
fig, ax = plt.subplots(ncols=4, figsize=(20,20))
for idx, image in enumerate(plot_images):
    ax[idx].imshow(image) 
plt.show()

# 3. Partition Unaugmented Data

## 3.1 MANUALLY SPLT DATA INTO TRAIN TEST AND VAL

In [ ]:
90*.7 # 63 to train

In [ ]:
90*.15 # 14 and 13 to test and val

## 3.2 Move the Matching Labels

In [34]:
for folder in ['train','test','val']:
    for file in os.listdir(os.path.join('data', folder, 'images')):
        
        filename = file.split('.')[0]+'.json'
        existing_filepath = os.path.join('data','labels', filename)
        if os.path.exists(existing_filepath): 
            new_filepath = os.path.join('data',folder,'labels',filename)
            os.replace(existing_filepath, new_filepath)